<a href="https://colab.research.google.com/github/MateusFreitas-C/Pesquisa-Operacional/blob/main/Aula_01.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Aula prática: Mix de Produção
<sup>Adaptado dos exercícios 2.3 e 2.5 do livro `Pesquisa Operacional`, de `Arenales, Armentano, Morabito e Yanasse`.</sup>

## Exercício 1

### Descrição do problema
Uma fundição tem de produzir 10 toneladas de um tipo de liga metálica e, para isso, tem disponível: lingotes de ferro, grafite e sucata. Dois componentes são relevantes para a liga: carbono e silício. As tabelas a seguir fornecem a fração, em termos percentuais, desses elementos nos ingredientes disponíveis, seus custos unitários, bem como a composição da liga (isto é, porcentagens mínima e máxima de cada componente da liga).

Frações dos elementos (%) nos ingredientes e custo dos ingredientes (R$/ton):

| | Lingotes | Grafite | Sucata |
|:---|:---:|:---:|:---:|
| Carbono | 0.5 | 90 | 9 |
| Silício | 14 | - | 27 |
| Custo | 90 | 180 | 25 |

Frações (%) mínima e máxima dos componentes na liga:

| | min | max |
|:---|:---:|:---:|
|Carbono | 0.0 | 9.5 |
|Silício | 19 | 20 |


Escreva um modelo de otimização linear para determinar as quantidades dos ingredientes para compor a liga metálica, de modo que as especificações técnicas sejam satisfeitas e o custo seja mínimo.

### Resolução

#### Resposta

$$ Min Z = 90 * X1 + 180 * X2 + 25 * X3 $$



S. a
$$ X1 + X2 + X3 = 10 $$
$$ 0\leq 0,5 * X1 + 90 * X2 + 9 * X3 \leq 9,5 $$
$$ 19\leq 14 * X1 + 0 * X2 + 27 * X3 \leq 20 $$
$$ X1, X2, X3 \geq 0 $$

In [ ]:
# instalação e importação do pacote mip
from mip import *

Carrega Dados

In [ ]:
# composição de cada ingrediente
a = {
    'l': {'c': 0.005, 's': 0.14},
    'g': {'c': 0.9,   's': 0.0},
    's': {'c': 0.09,  's': 0.27},
}

# custo
c = {'l': 90, 'g': 180, 's': 25}

# composições mínimas e máximas dos componentes
n = {'c': 0.0, 's': 0.19}  # min
m = {'c': 0.095, 's': 0.2} # max

# quantidade desejada da liga
Q = 10

Cria modelo

In [ ]:
modelo = Model(sense=MINIMIZE, solver_name=CBC)

#Criando variáveis
x = [modelo.add_var(var_type=CONTINUOUS, name=f"x_{i}", lb=0.0) for i in range(3)]

#Função objetivo
modelo.objective = c['l']*x[0] + c['g']*x[1] + c['s']*x[2]

#Quantidade total
modelo += x[0] + x[1] + x[2] == Q

#Restrição de carbono
carbono = a['l']['c']*x[0] + a['g']['c']*x[1] + a['s']['c']*x[2]

modelo += n['c']*Q <= carbono
modelo += carbono <= m['c']*Q

#Restrição de silicio
silicio = a['l']['s']*x[0] + a['g']['s']*x[1] + a['s']['s']*x[2]

modelo += n['s']*Q <= silicio
modelo += silicio <= m['s']*Q

modelo.write("model.lp") # salva modelo em arquivo
with open("model.lp") as f: # lê e exibe conteúdo do arquivo
  print(f.read())

\Problem name: 

Minimize
OBJROW: 90 x_0 + 180 x_1 + 25 x_2
Subject To
constr(0):  x_0 + x_1 + x_2 = 10
constr(1):  0.00500 x_0 + 0.90000 x_1 + 0.09000 x_2 >= -0
constr(2):  0.00500 x_0 + 0.90000 x_1 + 0.09000 x_2 <= 0.95000
constr(3):  0.14000 x_0 + 0.27000 x_2 >= 1.90000
constr(4):  0.14000 x_0 + 0.27000 x_2 <= 2
Bounds
End



Executa

In [ ]:
def solve(modelo):
  status = modelo.optimize()

  print("Status = ", status)
  print(f"Solution value  = {modelo.objective_value:.2f}\n")

  print("Solution:")
  for v in modelo.vars:
      print(f"{v.name} = {v.x:.2f}")

solve(modelo)

Status =  OptimizationStatus.OPTIMAL
Solution value  = 600.00

Solution:
x_0 = 5.38
x_1 = 0.00
x_2 = 4.62


## Exercício 2

Agora considere que os ingredientes tem o estoque limitado, de acordo com a tabela abaixo.

| | Lingotes | Grafite | Sucata |
|:---|:---:|:---:|:---:|
| Estoque (ton) | 5 | 5 | 12 |

Como o modelo pode ser modificado para atender a esse requisito?

### Resposta

$$ Min Z = 90 * X1 + 180 * X2 + 25 * X3 $$



S. a
$$ X1 + X2 + X3 = 10 $$
$$ 0\leq 0,5 * X1 + 90 * X2 + 9 * X3 \leq 9,5 $$
$$ 19\leq 14 * X1 + 0 * X2 + 27 * X3 \leq 20 $$
$$ X1, X2, X3 \geq 0 $$
$$ X1 \leq 5 $$
$$ X2 \leq 5 $$
$$ X3 \leq 12 $$

### Código

Carrega Dados

In [ ]:
# estoque
e = {'l': 5, 'g': 5, 's': 12}

Cria modelo

In [ ]:
modelo += x[0] <= e['l']
modelo += x[1] <= e['g']
modelo += x[2] <= e['s']

modelo.write("modelo2.lp") # salva modelo em arquivo
with open("modelo2.lp") as f: # lê e exibe conteúdo do arquivo
  print(f.read())

\Problem name: 

Minimize
OBJROW: 90 x_0 + 180 x_1 + 25 x_2
Subject To
constr(0):  x_0 + x_1 + x_2 = 10
constr(1):  0.00500 x_0 + 0.90000 x_1 + 0.09000 x_2 >= -0
constr(2):  0.00500 x_0 + 0.90000 x_1 + 0.09000 x_2 <= 0.95000
constr(3):  0.14000 x_0 + 0.27000 x_2 >= 1.90000
constr(4):  0.14000 x_0 + 0.27000 x_2 <= 2
constr(5):  x_0 <= 5
constr(6):  x_1 <= 5
constr(7):  x_2 <= 12
Bounds
End



Executa

In [ ]:
solve(modelo)

Status =  OptimizationStatus.OPTIMAL
Solution value  = 603.70

Solution:
x_0 = 5.00
x_1 = 0.19
x_2 = 4.81


## Exercício 3

Suponha agora que duas ligas metálicas devem ser preparadas e os mesmos ingredientes são utilizados em ambas. A liga especificada no Exercício 1 é referida como liga 1 e devem ser produzidas 10 toneladas desta liga. Da outra liga, referida como liga 2, devem ser produzidas 6 toneladas e suas composições mínima e máxima são dadas na tabela abaixo.

| | min | max |
|:---|:---:|:---:|
|Carbono | 0.00 | 40 |
|Silício | 12 | 19 |


### Resposta

$$ Min Z = (90 * X1 + 180 * X2 + 25 * X3) + (90 * Y1 + 180 * Y2 + 25 * Y3) $$



S. a
$$ X1 + X2 + X3 = 10 $$
$$ Y1 + Y2 + Y3 = 6 $$
$$ 0\leq 0,5 * X1 + 90 * X2 + 9 * X3 \leq 9,5 $$
$$ 0\leq 0,5 * Y1 + 90 * Y2 + 9 * Y3 \leq 40 $$
$$ 19\leq 14 * X1 + 0 * X2 + 27 * X3 \leq 20 $$
$$ 12\leq 14 * Y1 + 0 * Y2 + 27 * Y3 \leq 19 $$
$$ X1, X2, X3 \geq 0 $$
$$ X1 \leq 5 $$
$$ X2 \leq 5 $$
$$ X3 \leq 12 $$

### Código

Carrega dados

In [ ]:
# composições mínimas e máximas dos componentes
n = [{'c': 0.0, 's': 0.19}, {'c': 0.0, 's': 0.12}]
m = [{'c': 0.095, 's': 0.2}, {'c': 0.4, 's': 0.19}]

# quantidade desejada da liga
Q = [10, 6]

Cria modelo

In [ ]:
modelo = Model(sense=MINIMIZE, solver_name=CBC)

#Criando variáveis
x = [modelo.add_var(var_type=CONTINUOUS, name=f"x_{i}", lb=0.0) for i in range(3)]

y = [modelo.add_var(var_type=CONTINUOUS, name=f"y_{i}", lb=0.0) for i in range(3)]

#Função objetivo
modelo.objective = (c['l']*x[0] + c['g']*x[1] + c['s']*x[2]) + (c['l']*y[0] + c['g']*y[1] + c['s']*y[2])

#Restrição quantidade liga 1
modelo += x[0] + x[1] + x[2] == Q[0]

#Restrição quantidade liga 2
modelo += y[0] + y[1] + y[2] == Q[1]

#Restrição carbono liga 1
carbono = a['l']['c']*x[0] + a['g']['c']*x[1] + a['s']['c']*x[2]

modelo += n[0]['c']*Q[0] <= carbono
modelo += carbono <= m[0]['c']*Q[0]

#Restrição carbono liga 2
carbono = a['l']['c']*y[0] + a['g']['c']*y[1] + a['s']['c']*y[2]

modelo += n[1]['c']*Q[1] <= carbono
modelo += carbono <= m[1]['c']*Q[1]

#Restrição silicio liga 1
silicio = a['l']['s']*x[0] + a['g']['s']*x[1] + a['s']['s']*x[2]

modelo += n[0]['s']*Q[0] <= silicio
modelo += silicio <= m[0]['s']*Q[0]

#Restrição silicio liga 2
silicio = a['l']['s']*y[0] + a['g']['s']*y[1] + a['s']['s']*y[2]

modelo += n[1]['s']*Q[1] <= silicio
modelo += silicio <= m[1]['s']*Q[1]

#Restrição estoque
modelo += x[0] + y[0] <= e['l']
modelo += x[1] + y[1] <= e['g']
modelo += x[2] + y[2] <= e['s']

modelo.write("modelo3.lp") # salva modelo em arquivo
with open("modelo3.lp") as f: # lê e exibe conteúdo do arquivo
  print(f.read())

\Problem name: 

Minimize
OBJROW: 90 x_0 + 180 x_1 + 25 x_2 + 90 y_0 + 180 y_1 + 25 y_2
Subject To
constr(0):  x_0 + x_1 + x_2 = 10
constr(1):  y_0 + y_1 + y_2 = 6
constr(2):  0.00500 x_0 + 0.90000 x_1 + 0.09000 x_2 >= -0
constr(3):  0.00500 x_0 + 0.90000 x_1 + 0.09000 x_2 <= 0.95000
constr(4):  0.00500 y_0 + 0.90000 y_1 + 0.09000 y_2 >= -0
constr(5):  0.00500 y_0 + 0.90000 y_1 + 0.09000 y_2 <= 2.40000
constr(6):  0.14000 x_0 + 0.27000 x_2 >= 1.90000
constr(7):  0.14000 x_0 + 0.27000 x_2 <= 2
constr(8):  0.14000 y_0 + 0.27000 y_2 >= 0.72000
constr(9):  0.14000 y_0 + 0.27000 y_2 <= 1.14000
constr(10):  x_0 + y_0 <= 5
constr(11):  x_1 + y_1 <= 5
constr(12):  x_2 + y_2 <= 12
Bounds
End



Executa

In [ ]:
solve(modelo)

Status =  OptimizationStatus.OPTIMAL
Solution value  = 1029.26

Solution:
x_0 = 4.32
x_1 = 0.51
x_2 = 5.17
y_0 = 0.68
y_1 = 1.45
y_2 = 3.87
